In [2]:
import torch
import os
from pathlib import Path
from munch import Munch
from dataset import Dataset

# load dataset
data_path = Path('data')
data_files = {'en': [], 'id': []}

for name in data_path.iterdir():
    for lang in data_files.keys():
        lang_path = name / lang

        for dir in lang_path.iterdir():
            if dir.is_file():
                with open(dir, 'rb') as f:
                    content = f.read().decode('UTF-8')

                    data_files[lang].append(Munch(
                        name=str(dir.name),
                        path=str(dir),
                        content=content))

In [11]:
from collections import Counter
from nltk import sent_tokenize, word_tokenize
from tqdm import tqdm

import torch
import pickle

def build_dataset(docs):
    vocab_idx = Counter(['<bos>', '<eos>'])
    seq_words = []

    for doc in tqdm(docs, 'processed documents'):
        sentences = sent_tokenize(doc)
        
        for sent in sentences:
            words = word_tokenize(sent)

            seq_words.append(words)
            vocab_idx.update(words)
    
    vocab_idx = dict(vocab_idx)
    count = len(vocab_idx)

    vocab = dict()

    for i, k in enumerate(vocab_idx.keys()):
        ohe = [0] * count
        ohe[i] = 1

        vocab_idx[k] = i
        vocab[k] = ohe

    seq_tensors = []

    BOS_IDX = vocab['<bos>']
    EOS_IDX = vocab['<eos>']

    for words in seq_words:
        seq_tensors.append(torch.tensor([vocab[BOS_IDX]] + [vocab[w] for w in words]) + [vocab[EOS_IDX]])
        
    return Munch(vocab=vocab, data=seq_words, tensors=seq_tensors)

data_en = build_dataset([doc.content for doc in data_files['en']])
data_id = build_dataset([doc.content for doc in data_files['id']])

dataset = (data_en, data_id)

with open('dataset', 'wb') as f:
    pickle.dump(dataset, f)

processed documents: 100%|██████████| 20/20 [36:30<00:00, 109.54s/it]  


In [17]:

# from nltk import sent_tokenize, word_tokenize
# import codecs

# sentences = sent_tokenize(data_files['en'][0].content)
# words = word_tokenize(sentences[0])

# print(words)

# from torchtext.data.utils import get_tokenizer
# english_tokenizer = get_tokenizer('spacy', language='en')
# indonesian_tokenizer = get_tokenizer('spacy', language='in')

# def build_vocabulary(filepath, tokenizer):
#   counter = Counter()
#   with io.open(filepath, encoding="utf8") as f:
#     for string_ in f:
#       counter.update(tokenizer(string_))
#   return Vocab(counter, specials=['<unk>', '<pad>', '<bos>', '<eos>'])

# english_vocabularies = build_vocabulary(data_files['en'], )

'typeof(data_filesasdfsafdaf0].content)'

In [ ]:
from gensim.models import Word2Vec

gensim_model = Word2Vec(sentences=data_files['en'],
                        vector_size=100,
                        window=5,
                        min_count=1,
                        workers=4)

print(gensim_model["awesome"])

In [ ]:
class TransformerModel(nn.Module):
    def __init__(self, ntoken, d_model, nhead, d_hid,
                 nlayers, dropout):
        super().__init__()
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = TransformerEncoderLayer(d_model, nhead, d_hid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, d_model)
        self.d_model = d_model
        self.decoder = nn.Linear(d_model, ntoken)
        self.init_weights()

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src, src_mask):
        src = self.encoder(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, src_mask)
        output = self.decoder(output)
        return output


def generate_square_subsequent_mask(sz):
    return torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)